## Capstone Project (Week 3)

#### Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Scraping The Data

In [21]:
html_doc=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc.text, 'lxml')

### Creating a DataFrame

In [22]:
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    # Header
    if(index==0):
        columns=section
    else:
        data.append(section)
        
canada_df = pd.DataFrame(data = data, columns = columns)

### Cleaning The Data More And Making It more suitable for further Analysis

In [24]:
canada_df = canada_df[canada_df['Borough']!= 'Not assigned']
canada_df["Neighbourhood"] = canada_df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
canada_df = canada_df.drop_duplicates()

#update index to be postcode if it isn't already
#if(canada_df.index.name != 'Postcode'):
#   canada_df = canada_df.set_index('Postcode')

canada_df['Neighbourhood'].replace('Not assigned', canada_df['Borough'], inplace = True)
canada_df
neighbourhoods = canada_df
neighbourhoods = neighbourhoods.reset_index()
neighbourhoods = neighbourhoods.drop(columns = ['index'])
neighbourhoods.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"
